<a href="https://colab.research.google.com/github/romiebanerjee/CLIP-SAE/blob/master/CLIP_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
# !pip install -q transformers torch torchvision pillow
# !pip install -q git+https://github.com/openai/CLIP.git

from IPython.display import display

import torch
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model_names = ["openai/clip-vit-base-patch32", "openai/clip-vit-large-patch14", "laion/CLIP-ViT-bigG-14-laion2B-39B-b160k"]

Using device: cpu


In [ ]:
model_name = model_names[0]
model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)
tokenizer = CLIPTokenizer.from_pretrained(model_name)

In [ ]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities
print(probs)

tensor([[0.9949, 0.0051]], grad_fn=<SoftmaxBackward0>)


In [ ]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `hf-access` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticat

In [ ]:
# from datasets import load_dataset

# # Stream the dataset without downloading it entirely
# ds = load_dataset("jp1924/Laion400m-3", streaming=True, split="train")

# # Take a small subset (e.g., 1000 samples)
# # small_ds = ds.take(10000)

In [ ]:
from datasets import load_dataset

# # Stream from a single Parquet file (local or remote)
# ds_00000_of_00449 = load_dataset("parquet", data_files="https://huggingface.co/datasets/jp1924/Laion400m-3/resolve/main/data/train-00000-of-00449.parquet", streaming=True, split="train")
# ds_00001_of_00449 = load_dataset("parquet", data_files="https://huggingface.co/datasets/jp1924/Laion400m-3/resolve/main/data/train-00001-of-00449.parquet", streaming=True, split="train")
# ds_00002_of_00449 = load_dataset("parquet", data_files="https://huggingface.co/datasets/jp1924/Laion400m-3/resolve/main/data/train-00002-of-00449.parquet", streaming=True, split="train")
# ds_00003_of_00449 = load_dataset("parquet", data_files="https://huggingface.co/datasets/jp1924/Laion400m-3/resolve/main/data/train-00003-of-00449.parquet", streaming=True, split="train")
ds_00004_of_00449 = load_dataset("parquet", data_files="https://huggingface.co/datasets/jp1924/Laion400m-3/resolve/main/data/train-00004-of-00449.parquet", streaming=True, split="train")
ds_00005_of_00449 = load_dataset("parquet", data_files="https://huggingface.co/datasets/jp1924/Laion400m-3/resolve/main/data/train-00005-of-00449.parquet", streaming=True, split="train")

In [ ]:
for idx, sample in enumerate(ds_00005_of_00449):
    print(idx, sample)
    print(sample['caption'])
    display(sample['image'])
    if idx > 10:
        break

In [ ]:
# text = tokenizer.decode(tokenizer("hello Ich bin ")['input_ids'][:3])
# text.replace("<|startoftext|>", "").replace("<|endoftext|>", "").strip()
max_position_embs = model.text_model.config.max_position_embeddings

In [ ]:
def clip_text(text):

    tokens = tokenizer(text)['input_ids'][:max_position_embs-2]
    clipped_text = tokenizer.decode(tokens)
    # print(clipped_text)
    clipped_text = clipped_text.replace("<|startoftext|>", "").replace("<|endoftext|>", "").strip()
    return clipped_text

clip_text("a phot o dfds;sd flsdflsd flksjdfls dfjs jfdlksjd flsjfljsdf dslfdslj fldskjf ls")

'a phot o dfds ; sd flsdflsd flksjdfls dfjs jfdlksjd flsjfljsdf dslfdslj fldskjf ls'

In [ ]:
def get_image_embedding(input):
    if isinstance(input, str):
    # Check if it is a url
      image = Image.open(requests.get(input, stream=True).raw)
    # Check if it's a JPEG file (file-like object or bytes)
    elif hasattr(input, 'read') or isinstance(input, bytes):
        # You might want to add more checks here to verify it's a JPEG
        image = Image.open(input)
    # Check if it's a PIL Image object
    elif isinstance(input, Image.Image):
        image = input
    else:
        return "Unknown type"

    try:
      # image = Image.open(image)
      # display(image)
      inputs = processor(images=image, return_tensors="pt", padding=True).to(device)
      with torch.no_grad():
        outputs = model.vision_model(**inputs)
        outputs = outputs.pooler_output
        image_embeds = model.visual_projection(outputs)
        image_embeds = image_embeds / image_embeds.norm(dim=-1, keepdim=True)

        return image_embeds.cpu()
    except Exception as e:
        print(f"Error processing image: {image}")
        print(f"Error message: {str(e)}")
        return torch.zeros((1, 512)).to(device)

def get_text_embedding(text):
    text = clip_text(text)
    inputs = processor(text=text, return_tensors="pt", padding=True).to(device)
    try:
      with torch.no_grad():
        outputs = model.text_model(**inputs)
        outputs = outputs.pooler_output
        text_embeds = model.text_projection(outputs)
        text_embeds = text_embeds / text_embeds.norm(dim=-1, keepdim=True)
    except Exception as e:
        print(f"Error processing text: {text}")
        print(f"Error message: {str(e)}")
        return torch.zeros((1, 512)).to(device)

    return text_embeds.cpu()


In [ ]:
dataset_name = ds_00005_of_00449
from tqdm  import tqdm
image_embeds = []
text_embeds = []
for idx, sample in tqdm(enumerate(dataset_name)):
    # if idx > 1000:
    #     break
    image_emb = get_image_embedding(sample['image'])
    caption_emb = get_text_embedding(sample['caption'])
    image_embeds.append(image_emb)
    text_embeds.append(caption_emb)


155898it [4:15:59, 10.15it/s]


In [ ]:
image_embeddings = torch.stack(image_embeds)
text_embeddings = torch.stack(text_embeds)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

torch.save(image_embeddings, '/content/drive/MyDrive/data/CLIP_embeddings/image_embeddings_00005-00449.pt')
torch.save(text_embeddings, '/content/drive/MyDrive/data/CLIP_embeddings/text_embeddings_00005-00449.pt')